#**Actividad 7: Regresión logística**
##Daniel Cruz Arciniega - A01701370

## 1. Cargar el dataframe y analizarlo

In [102]:
import pandas as pd
import seaborn as sns

# Cargar el archivo CSV
df = pd.read_csv("/content/cuentas_credicel.csv", encoding="latin1")
df.head()

<ipython-input-102-a0ce1d307631>:5: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/cuentas_credicel.csv", encoding="latin1")


,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
0,3,HDNAWQ3A,30,12/11/21 19:00,ZTE,Blade A3 2020 RO B,26S,1949.0,780.0,0.0,...,40.02,5000,26,NaN,23.0,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02
1,4,M418YNR4,33,12/11/21 19:15,MOTOROLA,LTE XT2097-12 E7I POWER Azul,13S,2999.0,1050.0,0.0,...,35.01,6000,13,NaN,60.0,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07
2,5,SZ7V3NZT,37,13/11/21 14:13,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,0,74,NaN,58.0,TANTOYUCA,VER.,0,0,0
3,6,9I08UIQZ,38,13/11/21 15:45,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,7000,13,NaN,42.0,AMXTLAN PUE,NaN,Zacatlan,Puebla,SAMJ790115MPLNRS02
4,7,JMF4LKYU,44,13/11/21 16:10,MOTOROLA,G10 B,39S,4299.0,1075.0,0.0,...,25.01,7000,74,NaN,33.0,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07


In [103]:
df.dtypes

folio                   int64
tag                    object
folio_solicitud         int64
fecha                  object
marca                  object
modelo                 object
plazo                  object
precio                float64
enganche              float64
descuento             float64
semana                  int64
monto_financiado      float64
costo_total             int64
monto_accesorios      float64
agente_venta           object
dis_venta              object
status                  int64
fraude                  int64
empresa                object
inversion               int64
pagos_realizados        int64
reautorizacion          int64
fecha_ultimo_pago      object
fecha_pago_proximo     object
status_cuenta          object
puntos                  int64
riesgo                float64
score_buro              int64
razones_buro           object
porc_eng              float64
limite_credito         object
semana_actual           int64
cp_cliente             object
edad_clien

In [104]:
df['limite_credito'] = df['limite_credito'].str.extract('(\d+\.\d+|\d+)').astype(float)

In [105]:
numeric = df.select_dtypes(include=['float64', 'int64']).columns
df = df[numeric]

##2. Valores nulos

In [106]:
df.isna().sum()

folio                  0
folio_solicitud        0
precio                 0
enganche               0
descuento              0
semana                 0
monto_financiado       0
costo_total            0
monto_accesorios       0
status                 0
fraude                 0
inversion              0
pagos_realizados       0
reautorizacion         0
puntos                 0
riesgo                 0
score_buro             0
porc_eng               0
limite_credito      6353
semana_actual          0
edad_cliente           9
dtype: int64

In [107]:
cols = [
    'edad_cliente',
    'limite_credito'
]

for col in cols:
  median_val = df[col].median()
  df[col] = df[col].fillna(median_val)

df.isna().sum()

<ipython-input-107-1ce7d3a05504>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(median_val)
<ipython-input-107-1ce7d3a05504>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(median_val)


folio               0
folio_solicitud     0
precio              0
enganche            0
descuento           0
semana              0
monto_financiado    0
costo_total         0
monto_accesorios    0
status              0
fraude              0
inversion           0
pagos_realizados    0
reautorizacion      0
puntos              0
riesgo              0
score_buro          0
porc_eng            0
limite_credito      0
semana_actual       0
edad_cliente        0
dtype: int64

## 3. Outliers

In [108]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7653 entries, 3296 to 22732
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   folio             7653 non-null   int64  
 1   folio_solicitud   7653 non-null   int64  
 2   precio            7653 non-null   float64
 3   enganche          7653 non-null   float64
 4   descuento         7653 non-null   float64
 5   semana            7653 non-null   int64  
 6   monto_financiado  7653 non-null   float64
 7   costo_total       7653 non-null   int64  
 8   monto_accesorios  7653 non-null   float64
 9   status            7653 non-null   int64  
 10  fraude            7653 non-null   int64  
 11  inversion         7653 non-null   int64  
 12  pagos_realizados  7653 non-null   int64  
 13  reautorizacion    7653 non-null   int64  
 14  puntos            7653 non-null   int64  
 15  riesgo            7653 non-null   float64
 16  score_buro        7653 non-null   int6

## 4. 10 casos de correlación logística

In [109]:
# Categorizar los valores
def cat_value(val):
    p_50 = df['score_buro'].quantile(0.5)
    if val <= p_50:
        return 'bajo'
    else:
        return 'alto'

# Convertir score_buro a valores categóricos
df['score_buro'] = df['score_buro'].apply(cat_value)

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Seleccionar las variables independientes a analizar
xcols = ['precio',
        'costo_total',
        'enganche',
        'monto_financiado',
        'monto_accesorios',
        'pagos_realizados',
        'reautorizacion',
        'limite_credito',
        'descuento',
        'edad_cliente']

X = df[['precio',
        'costo_total',
        'enganche',
        'monto_financiado',
        'monto_accesorios',
        'pagos_realizados',
        'reautorizacion',
        'limite_credito',
        'descuento',
        'edad_cliente']]
y = df['score_buro']
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42)

# Iterar a través de las columnas y ajustar modelos de regresión logística
res = pd.DataFrame(columns=['Variable', 'Exactitud', 'Precisión', 'F1'])
for col in X.columns:
    escalar = StandardScaler()
    X_train_col = X_train[[col]]
    X_test_col = X_test[[col]]
    X_train_col = escalar.fit_transform(X_train_col)
    X_test_col = escalar.transform(X_test_col)

    # Declarar y entrenar el modelo
    model = LogisticRegression()
    model.fit(X_train_col, y_train)
    y_pred = model.predict(X_test_col)

    mat = confusion_matrix(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    pres = precision_score(y_test, y_pred, average="binary", pos_label="alto")
    f1 = f1_score(y_test, y_pred, average="binary", pos_label="alto")


    # Guardar los resultados
    res = res.append({'Variable': col,
                      'Exactitud': acc,
                      'Precisión': pres,
                      'F1': f1},
                      ignore_index=True)

    # Imprimir matriz de confusión
    print(f"\nMatriz de Confusión de la columna {col}")
    print(mat)

<ipython-input-110-e62e12be7f6b>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': col,
<ipython-input-110-e62e12be7f6b>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': col,
<ipython-input-110-e62e12be7f6b>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': col,



Matriz de Confusión de la columna precio
[[442 693]
 [393 768]]

Matriz de Confusión de la columna costo_total
[[517 618]
 [417 744]]

Matriz de Confusión de la columna enganche
[[697 438]
 [585 576]]


<ipython-input-110-e62e12be7f6b>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': col,
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-110-e62e12be7f6b>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': col,
<ipython-input-110-e62e12be7f6b>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': col,
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefin


Matriz de Confusión de la columna monto_financiado
[[472 663]
 [395 766]]

Matriz de Confusión de la columna monto_accesorios
[[   0 1135]
 [   0 1161]]

Matriz de Confusión de la columna pagos_realizados
[[486 649]
 [408 753]]

Matriz de Confusión de la columna reautorizacion
[[   0 1135]
 [   0 1161]]

Matriz de Confusión de la columna limite_credito
[[867 268]
 [602 559]]

Matriz de Confusión de la columna descuento
[[312 823]
 [281 880]]

Matriz de Confusión de la columna edad_cliente
[[553 582]
 [458 703]]


<ipython-input-110-e62e12be7f6b>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': col,
<ipython-input-110-e62e12be7f6b>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': col,
<ipython-input-110-e62e12be7f6b>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': col,


## 5. Resultados

In [111]:
res

,Variable,Exactitud,Precisión,F1
0,precio,0.527003,0.529341,0.448731
1,costo_total,0.549216,0.553533,0.499758
2,enganche,0.554443,0.543682,0.576748
3,monto_financiado,0.539199,0.544406,0.471528
4,monto_accesorios,0.505662,0.000000,0.000000
5,pagos_realizados,0.539634,0.543624,0.479054
6,reautorizacion,0.505662,0.000000,0.000000
7,limite_credito,0.621080,0.590197,0.665899
8,descuento,0.519164,0.526138,0.361111
9,edad_cliente,0.547038,0.546983,0.515377


In [112]:
res.describe()

,Exactitud,Precisión,F1
count,10.000000,10.000000,10.000000
mean,0.540810,0.437791,0.401821
std,0.033120,0.231381,0.226182
min,0.505662,0.000000,0.000000
25%,0.521124,0.526939,0.383016
50%,0.539416,0.543653,0.475291
75%,0.548672,0.546339,0.511473
max,0.621080,0.590197,0.665899
